In [32]:
from container import Container
from template import Template, Example
api_key = 'OpenAI-Key'
model = Container(api_key, 'text-davinci-003')

In [33]:
import pandas as pd
from dataload import parse_data
train = pd.read_json('train.json')
test = pd.read_json('example.json')
one_shot_train = train.loc[train['pairs'].apply(len) == 1].sample(1)
five_shot_train = train.loc[train['pairs'].apply(len) == 1].sample(3).append(
    train.loc[train['pairs'].apply(len) == 2].sample(1)).append(
    train.loc[train['pairs'].apply(len) == 3].sample(1))

/var/folders/sb/nbhxgzzj2ds2h3yjxjyn8f2w0000gn/T/ipykernel_13306/1599189418.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_shot_train = train.loc[train['pairs'].apply(len) == 1].sample(3).append(
/var/folders/sb/nbhxgzzj2ds2h3yjxjyn8f2w0000gn/T/ipykernel_13306/1599189418.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  five_shot_train = train.loc[train['pairs'].apply(len) == 1].sample(3).append(


In [34]:
parse_train_1 = parse_data(one_shot_train)
parse_train_5 = parse_data(five_shot_train)
# parse_train_6 = parse_data(ten_shot_train)
parse_test = parse_data(test)

In [35]:
question_temp = '''Extract emotion-cause pairs and return a list of tuple based on the given content.
In each tuple, the first element is the index of the emotion clause and the second element is the index of cause clause.'''

question_temp_pair_num_limit = '''Extract 1 to 4 emotion-cause pairs and return a list of tuple based on the given content.
In each tuple, the first element is the index of the emotion clause and the second element is the index of cause clause.'''

question_temp_sequence_prompt = '''Extract emotion-cause pairs and return a list of tuple based on the given content.
In each tuple, the first element is the index of the emotion clause and the second element is the index of cause clause.
The cause clause usually appears in the front of emotion clause'''

question_temp_both_hint = '''Extract 1 to 4 emotion-cause pairs and return a list of tuple based on the given content.
In each tuple, the first element is the index of the emotion clause and the second element is the index of cause clause.
The cause clause usually appears in the front of emotion clause'''

DATA = parse_train_5
FILENAME = 'prompt_5_shot.txt'
RESULTNAME = 'result_5_shot.txt'
QUESTION = question_temp_both_hint

In [36]:
few_shot = [{'content': e['content'], 'pair': e['pair']} for e in DATA]
dev = [{'content': e['content'], 'pair': e['pair']} for e in parse_test]

In [37]:
instruction = "Think step by step. Give a response to the question in this format"
question = ['Question:\n', '{$ Question needs answer}']
train = ['Examples:\n', '[{$ Several representative examples of the given question with answers}]']
content = ['Content:\n', '{$ Passage with index which includes the emotion-cause pairs for Task 4}']

template = Template(instruction=instruction)
template(question=question, train=train, content=content)
# template(question=question, content=content)

[['Question:\n', '{$ Question needs answer}'],
 ['Examples:\n',
  '[{$ Several representative examples of the given question with answers}]'],
 ['Content:\n',
  '{$ Passage with index which includes the emotion-cause pairs for Task 4}']]

In [38]:
def few_shot_ecpe(model, template, content, temp=0.7, n=2, question=QUESTION):
    train = few_shot[:10]
    example = Example(question=question, train=train, content=content)
    completion, prompt = model.chat(template, example, temp, n)
    return completion, prompt

In [39]:
from tqdm import tqdm
prompts = []
predict = []
result = []
for i in tqdm(range(100)):
    completion, prompt = few_shot_ecpe(model, template, dev[i]['content'], temp=0.3, n=1)
    predict.append(completion.choices[0].text)
    prompts.append(prompt)
    result.append(dev[i]['pair'])

100%|██████████| 100/100 [03:52<00:00,  2.33s/it]


In [41]:
def parse_to_list(data):
    result = []
    for i in range(len(data)):
        predicted = data[i].replace('(', '[').replace(')', ']') #replace all () to []
        try:
            predicted = predicted.split(':')[1]
        except:
            pass
        predicted = predicted.strip() # strip all begining space
        predicted = eval(predicted)
        result.append(predicted)
    return result

predict = parse_to_list(predict)

In [42]:
def compare(Y1, Y2):
    '''Y1: Y_hat case, Y2: Y case'''
    correct = 0
    for c in Y2:
        if c in Y1:
            correct += 1
    return correct, len(Y1), len(Y2)

def calc_precision(Y_hat, Y):
    assert len(Y_hat) == len(Y), print('shape mismatched')
    correct = 0
    total = 0
    ground_true = 0
    for i in range(len(Y_hat)):
        c, t, a = compare(Y_hat[i], Y[i])
        correct += c
        total += t
        ground_true += a
    precision = correct/total
    recall = correct/ground_true
    f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, f1, correct, total, ground_true

calc_precision(predict, result)


(0.0954356846473029, 0.2, 0.12921348314606743, 23, 241, 115)

In [44]:
with open(FILENAME, 'w') as f:
    for sentence in prompts:
        f.write(sentence + '---------------End Case---------------\n\n')

with open(RESULTNAME, 'w') as f:
    for index in range(len(predict)):
        res = predict[index]
        re = result[index]
        f.write(str(res) + '---------------True---------------' + str(re) + '\n\n')